In [ ]:
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import train_test_split
from sklearn import metrics
import numpy as np
import torch.utils.data

In [ ]:
df1=pd.read_csv('opel_corsa_01.csv', sep=';',decimal=',')
df2=pd.read_csv('opel_corsa_02.csv', sep=';',decimal=',')
df3=pd.read_csv('peugeot_207_01.csv', sep=';',decimal=',')
df4=pd.read_csv('peugeot_207_02.csv', sep=';',decimal=',')
df_Dataset=pd.concat([df1,df2,df3,df4],axis=0)
df_Dataset.index

In [ ]:
if (df_Dataset.isnull().sum().sum()!=0):
    df_Dataset.dropna(axis=0, how='any', thresh=None, subset=None, inplace=True)
df_Dataset.index

In [ ]:
y = df_Dataset['drivingStyle']
X = df_Dataset.drop(['drivingStyle','traffic','roadSurface'], axis=1)

train_X, test_X, train_y, test_y = train_test_split(X, y, test_size=0.2, random_state=0)

y.shape
X.shape

In [ ]:
from sklearn.preprocessing import MinMaxScaler

feature_scaler = MinMaxScaler()
train_X = feature_scaler.fit_transform(train_X)
test_X = feature_scaler.transform(test_X)

In [ ]:
Xtrain, Xval, Ytrain, Yval = train_test_split(train_X, train_y, test_size=0.2, random_state=5)

In [ ]:
print(train_X)

In [ ]:
train_X.shape

In [ ]:
train_y[train_y=='AggressiveStyle']=0
train_y[train_y=='EvenPaceStyle']=1
train_y


In [ ]:
test_y[test_y=='AggressiveStyle']=0
test_y[test_y=='EvenPaceStyle']=1
test_y

Запуск Duet

In [ ]:
import syft as sy
duet = sy.launch_duet()
sy.logging(file_path="./syft_do.log")

In [ ]:
import torch

# determine the supported device
def get_device():
    if torch.cuda.is_available():
        device = torch.device('cuda:0')
    else:
        device = torch.device('cpu') # don't have GPU
    return device

# convert a df to tensor to be used in pytorch
def df_to_tensor(df):
    device = get_device()
    return torch.from_numpy(df).float().to(device)

Загрузка тензоров в Duet store

In [ ]:
X_train = df_to_tensor(train_X)
torch.Tensor.tag(X_train, 'Train X')
torch.Tensor.send(X_train, duet, searchable=True)

Y_train = df_to_tensor(train_y.to_numpy(dtype=np.float64))
torch.Tensor.tag(Y_train, 'Train Y')
torch.Tensor.send(Y_train, duet, searchable=True)

X_test = df_to_tensor(test_X)
torch.Tensor.tag(X_test, 'Test X')
torch.Tensor.send(X_test, duet, searchable=True)

Y_test = df_to_tensor(test_y.to_numpy(dtype=np.float64))
torch.Tensor.tag(X_test, 'Test Y')
torch.Tensor.send(X_test, duet, searchable=True)

In [ ]:
train_num = sy.lib.python.Int(X_train.shape[0])
train_num.tag("train_num")
train_num.send(duet, searchable=True)

In [ ]:
duet.store.pandas

In [ ]:
duet.requests.pandas

In [ ]:
duet.requests[0].accept()

In [ ]:
duet.requests.add_handler(
    name="cuda_is_available",
    action="accept"
)

duet.requests.add_handler(
    name="loss",
    action="accept",
    timeout_secs=-1,  # no timeout
    print_local=True  # print the result in your notebook
)

duet.requests.add_handler(
    name="train_size",
    action="accept"
)

duet.requests.add_handler(
    name="inference",
    action="accept"
)

duet.requests.add_handler(
    name="model_download",
    action="accept"
)

In [ ]:
duet.requests.handlers